In [ ]:
# 安装环境
using Pkg
Pkg.activate(".")
# Pkg.add("DataFrames")
# Pkg.add("PyCall")
Pkg.instantiate()

In [2]:
using CSV, LightXML, DataFrames, PyCall

In [3]:
@pyimport deep_translator

function cn(text)
    if text == "International"
        "国际"
    elseif text == "European Union"
        "欧盟"
    elseif text == "National Cryosphere Desert Data Center"
        "国家冰川冻土沙漠科学数据中心"
    else
        deep_translator.GoogleTranslator(source="auto", target="zh-CN").translate(text)
    end
end

cn (generic function with 1 method)

In [4]:
@pyimport pycountry

function country(alpha_3)
    if alpha_3 == "AAA"
        "International"
    elseif alpha_3 == "EEC"
        "European Union"
    else
        pycountry.countries.get(alpha_3=alpha_3).name
    end
end

country (generic function with 1 method)

In [ ]:
# country("AAA")

In [5]:
root_path = joinpath(dirname(@__FILE__))

"/home/oflg/Code/ucas/论文/雪冰科学数据中心"

In [6]:
df = CSV.read(joinpath(root_path, "data", "snow_ice_data.csv"), DataFrame)
metas = df[!, :metadata];

In [11]:
infos = []

for meta in metas
    # 名称、网址、所属机构、所在国家或国际组织、数据中心简介、特色及特有资源。
    xml = parse_string(meta)
    xml_root = root(xml)
    repo = find_element(xml_root, "repository")
    orgIdentifier = content(find_element(repo, "re3data.orgIdentifier"))
    repositoryName = content(find_element(repo, "repositoryName"))
    repositoryNameCn = cn(repositoryName)
    repositoryURL = content(find_element(repo, "repositoryURL"))
    description = content(find_element(repo, "description"))
    descriptionCn = cn(description)

    if occursin("<<<", description)
        continue
    end

    # 国家、机构相关数据
    institutions_xml = get_elements_by_tagname(repo, "institution")
    institutions_array = []
    institutionsCn_array = []
    countries_array = []
    countriesCn_array = []
    println(repositoryNameCn)
    
    for inst in institutions_xml
        institutionName = content(find_element(inst, "institutionName"))
        institutionNameCn = cn(institutionName)
        institutionCountry = country(content(find_element(inst, "institutionCountry")))
        institutionCountryCn = cn(institutionCountry)
        inst = "$institutionName($institutionCountry)"
        instCn = "$institutionNameCn($institutionCountryCn)"
        push!(countries_array, institutionCountry)
        push!(countriesCn_array, institutionCountryCn)
        push!(institutions_array, inst)
        push!(institutionsCn_array, instCn)
    end


    if "International" in countries_array || "European Union" in countries_array
        continue
    end

    countries = join(unique(countries_array), "; ")
    countriesCn = join(unique(countriesCn_array), "; ")
    institutions = join(institutions_array, "; ")
    institutionsCn = join(institutionsCn_array, "; ")
    contentType_xml = get_elements_by_tagname(repo, "contentType")
    contentTypes_array = []
    contentTypesCn_array = []

    for cont in contentType_xml
        contentType = content(cont)
        contentTypeCn = cn(contentType)
        push!(contentTypes_array, contentType)
        push!(contentTypesCn_array, contentTypeCn)
    end

    contentTypes = join(contentTypes_array, "; ")
    contentTypesCn = join(contentTypesCn_array, "; ")


    # 将元数据添加到数组中
    push!(infos, (
        orgIdentifier=orgIdentifier,
        repositoryName=repositoryName,
        repositoryNameCn=repositoryNameCn,
        repositoryURL=repositoryURL,
        countries=countries,
        countriesCn=countriesCn,
        institutions=institutions,
        institutionsCn=institutionsCn,
        contentTypes=contentTypes,
        contentTypesCn=contentTypesCn,
        description=description,
        descriptionCn=descriptionCn,
    ))
    # break
end

# text
# 将数组写入csv文件
CSV.write(joinpath(root_path, "data", "snow_ice_info.csv"), infos)

国家冰雪数据中心


开放测高


NASA 国家冰雪数据中心分布式主动档案中心


先进的北极合作数据和信息服务


剑桥世界冰川学数据中心


...

TerraSAR-X TOR


世界气候数据中心


专业的


国际Argo计划


"/home/oflg/Code/ucas/论文/雪冰科学数据中心/data/snow_ice_info.csv"